In [1]:
import string
import numpy as np
import pandas as pd
!pip install keras
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
dataset="CoronaVirus.txt"
dataset

'CoronaVirus.txt'

In [3]:
text=open(dataset).read().lower()
#dataset=dataset.lower()
print("corpus length:", len(text))

corpus length: 949


In [4]:
def remove_punctuation(sentence):
    return sentence.translate(str.maketrans('','', string.punctuation))

In [5]:

def add2dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)
   

In [6]:
def list2probabilitydict(given_list):
    probability_dict = {}
    given_list_length = len(given_list)
    for item in given_list:
        probability_dict[item] = probability_dict.get(item, 0) + 1
    for key, value in probability_dict.items():
        probability_dict[key] = value / given_list_length
    return probability_dict


In [7]:
first_word = {}
second_word = {}
transitions = {}


In [15]:
print(first_word)
print(second_word)

{'coronavirus': 0.25, 'most': 0.25, 'the': 0.5}
{'coronavirus': {'disease': 1.0}, 'most': {'people': 1.0}, 'the': {'best': 0.5, 'covid19': 0.5}}


In [16]:
# the probabilities of moving from one state to another in a dynamic system.
print(transitions)

{('coronavirus', 'disease'): {'covid19': 1.0}, ('disease', 'covid19'): {'is': 1.0}, ('covid19', 'is'): {'an': 1.0}, ('is', 'an'): {'infectious': 1.0}, ('an', 'infectious'): {'disease': 1.0}, ('infectious', 'disease'): {'caused': 1.0}, ('disease', 'caused'): {'by': 1.0}, ('caused', 'by'): {'a': 1.0}, ('by', 'a'): {'newly': 1.0}, ('a', 'newly'): {'discovered': 1.0}, ('discovered', 'coronavirus'): {'END': 1.0}, ('newly', 'discovered'): {'coronavirus': 1.0}, ('most', 'people'): {'infected': 1.0}, ('people', 'infected'): {'with': 1.0}, ('infected', 'with'): {'the': 1.0}, ('with', 'the'): {'covid19': 1.0}, ('the', 'covid19'): {'virus': 1.0}, ('covid19', 'virus'): {'will': 0.3333333333333333, 'the': 0.3333333333333333, 'spreads': 0.3333333333333333}, ('virus', 'will'): {'experience': 1.0}, ('will', 'experience'): {'mild': 1.0}, ('experience', 'mild'): {'to': 1.0}, ('mild', 'to'): {'moderate': 1.0}, ('to', 'moderate'): {'respiratory': 1.0}, ('moderate', 'respiratory'): {'illness': 1.0}, ('resp

In [8]:
# Trains a Markov model based on the data in dataset
def train_markov_model():
    for line in open(dataset):
        tokens = remove_punctuation(line.rstrip().lower()).split()
        tokens_length = len(tokens)
        for i in range(tokens_length):
            token = tokens[i]
            if i == 0:
                first_word[token] = first_word.get(token, 0) + 1
            else:
                prev_token = tokens[i - 1]
                if i == tokens_length - 1:
                    add2dict(transitions, (prev_token, token), 'END')
                if i == 1:
                    add2dict(second_word, prev_token, token)
                else:
                    prev_prev_token = tokens[i - 2]
                    add2dict(transitions, (prev_prev_token, prev_token), token)
    
    # Normalize the distributions
    first_word_total = sum(first_word.values())
    for key, value in first_word.items():
        first_word[key] = value / first_word_total
        
    for prev_word, next_word_list in second_word.items():
        second_word[prev_word] = list2probabilitydict(next_word_list)
        
    for word_pair, next_word_list in transitions.items():
        transitions[word_pair] = list2probabilitydict(next_word_list)
    
    print('Training successful.')
   
    
    

In [9]:
train_markov_model()

Training successful.


In [10]:
def sample_word(dictionary):
    p0 = np.random.random()
    cumulative = 0
    for key, value in dictionary.items():
        cumulative += value
        if p0 < cumulative:
            return key
    assert(False)

In [11]:
number_of_sentences = 10

In [12]:
# Function to generate sample text
def generate():
    for i in range(number_of_sentences):
        sentence = []
        # Initial word
        word0 = sample_word(first_word)
        sentence.append(word0)
        # Second word
        word1 = sample_word(second_word[word0])
        sentence.append(word1)
        # Subsequent words untill END
        while True:
            word2 = sample_word(transitions[(word0, word1)])
            if word2 == 'END':
                break
            sentence.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(sentence))
       # test_word = sentence[2]
       # print(word2)

#print(test_word)

In [13]:
generate()

coronavirus disease covid19 is an infectious disease caused by a newly discovered coronavirus
most people infected with the covid19 virus spreads primarily through droplets of saliva or discharge from the nose when an infected person coughs or sneezes so it’s important that you also practice respiratory etiquette for example by coughing into a flexed elbow
the covid19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment older people and those with underlying medical problems like cardiovascular disease diabetes chronic respiratory disease and cancer are more likely to develop serious illness
coronavirus disease covid19 is an infectious disease caused by a newly discovered coronavirus
coronavirus disease covid19 is an infectious disease caused by a newly discovered coronavirus
the best way to prevent and slow down transmission is to be well informed about the covid19 virus will experience mild to moderate respiratory illness and reco